# Data validation

## Setup

In [1]:
library(tidyverse)
library(arrow)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘arrow’


The following object is masked from ‘package:readr’:

    read_table


The following object is masked from ‘package:utils’:

    timestamp




In [2]:
na_summary <- read_feather("data/na_summary_preprocessed.feather")
na_summary_original <- read_feather("data/na_summary.feather")

head(na_summary)

boardname,ned,gender,boardid,directorid,timebrd,annualreportdate,totalcompensation,eqlinkremratio,isin,genderratio,nationalitymix,numberdirectors,stdevage,cusip
<chr>,<lgl>,<chr>,<dbl>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
EQUITY ONE INC (De-listed 03/2017),TRUE,M,10925,1,3.9,2014-12-01,NA,NA,US2947521009,0.778,0.4,9,8.5,294752100
EQUITY ONE INC (De-listed 03/2017),TRUE,M,10925,1,2.9,2013-12-01,NA,NA,US2947521009,0.778,0.4,9,7.7,294752100
EQUITY ONE INC (De-listed 03/2017),TRUE,M,10925,1,0.9,2011-12-01,NA,NA,US2947521009,0.900,0.4,10,6.9,294752100
EQUITY ONE INC (De-listed 03/2017),TRUE,M,10925,1,4.9,2015-12-01,NA,NA,US2947521009,0.778,0.4,9,8.5,294752100
EQUITY ONE INC (De-listed 03/2017),TRUE,M,10925,1,1.9,2012-12-01,NA,NA,US2947521009,0.800,0.5,10,7.3,294752100
NATIONAL MEDICAL HEALTH CARD SYSTEMS INC (De-listed 04/2008),TRUE,M,21616,16,1.5,2006-06-01,NA,NA,US6369183024,1.000,0.0,10,9.4,636918302


## Check BoardEx dataset

### Annual report inconsistencies

In [3]:
na_summary %>%
    select(isin, annualreportdate, boardid) %>%
    group_by(isin, annualreportdate) %>%
    summarize(n = n_distinct(boardid)) %>%
    filter(n > 1)

`summarise()` regrouping output by 'isin' (override with `.groups` argument)



isin,annualreportdate,n
<chr>,<date>,<int>
US30224P2002,2013-12-01,2
US30224P2002,2014-12-01,2
US30224P2002,2015-12-01,2
US30224P2002,2016-12-01,2
US30224P2002,2017-12-01,2
US30224P2002,2018-12-01,2


In [4]:
na_summary %>%
    filter(isin == "US30224P2002" & annualreportdate == "2013-12-01") %>%
    select(isin, annualreportdate, boardid, everything()) %>%
    head()

isin,annualreportdate,boardid,boardname,ned,gender,directorid,timebrd,totalcompensation,eqlinkremratio,genderratio,nationalitymix,numberdirectors,stdevage,cusip
<chr>,<date>,<dbl>,<chr>,<lgl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
US30224P2002,2013-12-01,2009179,ESH Hospitality Inc (ESH Hospitality LLC prior to 11/2013),TRUE,M,36881,0.1,NA,NA,1,0,5,11.8,30224P200
US30224P2002,2013-12-01,2009180,EXTENDED STAY AMERICA INC,TRUE,M,36881,0.1,NA,NA,1,0,7,9.0,30224P200
US30224P2002,2013-12-01,2009180,EXTENDED STAY AMERICA INC,TRUE,M,90572,0.1,NA,NA,1,0,7,9.0,30224P200
US30224P2002,2013-12-01,2009179,ESH Hospitality Inc (ESH Hospitality LLC prior to 11/2013),TRUE,M,90572,0.1,NA,NA,1,0,5,11.8,30224P200
US30224P2002,2013-12-01,2009180,EXTENDED STAY AMERICA INC,FALSE,M,203140,0.1,NA,NA,1,0,7,9.0,30224P200
US30224P2002,2013-12-01,2009180,EXTENDED STAY AMERICA INC,TRUE,M,444845,0.1,NA,NA,1,0,7,9.0,30224P200


Looks like there is something funky going on with this one observation. Let's drop it.

Drop offending observation

In [5]:
na_summary <- filter(na_summary,isin != "US30224P2002")

### Check other variables

In [6]:
na_summary %>%
    select(isin, annualreportdate) %>%
    n_distinct()

na_summary %>%
    select(isin, annualreportdate, boardid, numberdirectors, nationalitymix, genderratio) %>%
    n_distinct()

[1] 96734

[1] 96734

Looks good

### Check for duplicate observations with different data

In [7]:
nrow(unique(select(na_summary, annualreportdate, isin, directorid)))

[1] 808601

In [8]:
nrow(unique(na_summary))

[1] 808847

There are 94 observations with divergences.

In [9]:
inconsistencies <- na_summary %>%
    group_by(isin, annualreportdate, directorid) %>%
    distinct() %>%
    mutate(n = n()) %>%
    filter(n > 1)

nrow(inconsistencies)

[1] 488

We have successfully identified all diverging duplicate observations.

In [10]:
head(select(inconsistencies, isin, annualreportdate, directorid, everything()))

isin,annualreportdate,directorid,boardname,ned,gender,boardid,timebrd,totalcompensation,eqlinkremratio,genderratio,nationalitymix,numberdirectors,stdevage,cusip,n
<chr>,<date>,<dbl>,<chr>,<lgl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>
US52186N1063,2006-12-01,3708,LEAPFROG ENTERPRISES INC (De-listed 04/2016),TRUE,M,69432,9.3,NA,NA,1.000,0.0,8,3.8,52186N106,2
US52186N1063,2006-12-01,3708,LEAPFROG ENTERPRISES INC (De-listed 04/2016),FALSE,M,69432,9.3,NA,NA,1.000,0.0,8,3.8,52186N106,2
US9497461015,2000-12-01,11522,WELLS FARGO & CO,TRUE,M,33264,19.9,0,NA,0.737,0.1,19,4.4,949746101,2
US9497461015,2000-12-01,11522,WELLS FARGO & CO,TRUE,M,33264,19.9,NA,NA,0.737,0.1,19,4.4,949746101,2
US36866W1062,2007-12-01,15177,GEMSTAR TV GUIDE INTERNATIONAL INC (De-listed 05/2008),TRUE,F,12961,3.0,NA,NA,0.778,0.0,9,9.2,36866W106,2
US36866W1062,2007-12-01,15177,GEMSTAR TV GUIDE INTERNATIONAL INC (De-listed 05/2008),TRUE,F,12961,3.0,0,NA,0.778,0.0,9,9.2,36866W106,2


Looks like the issue might be missing values. If we fill them in, we might be able to resolve this problem.

In [11]:
# In theory, we should be able to just arrange by variables and have NAs at the end, but for some reason it's not quite working.
inconsistencies$nas <- rowSums(is.na(inconsistencies))

inconsistencies <- inconsistencies %>%
    group_by(isin, annualreportdate, directorid) %>%
    arrange(nas) %>%
    fill(totalcompensation, eqlinkremratio) %>%
    select(-c(n, nas))

nrow(inconsistencies)

[1] 488

In [12]:
inconsistencies_remaining <- inconsistencies %>%
    group_by(isin, annualreportdate, directorid) %>%
    distinct() %>%
    mutate(n = n()) %>%
    filter(n > 1)
nrow(inconsistencies_remaining)

[1] 400

This has resolved problems for 44 observations, but the remaining ones we cannot resolve. The problem is probably data that was later updated (e.g., NAs filled in), but because we do not have a date of update, we do not know which data is more recent.

### Multiple annual reports in a year

If there are multple annual reports in any year, the number of entries should be reduced when we extract only the year from the annual report date column.

In [13]:
na_summary %>%
    select(isin, annualreportdate) %>%
    n_distinct()

[1] 96734

In [14]:
na_summary %>%
    mutate(year = lubridate::year(annualreportdate)) %>%
    select(isin, year) %>%
    n_distinct()

[1] 96099

There 635 instances of duplicate company-year observation, meaning 1270 observations to be removed.

We could drop these, too.

In [15]:
# na_summary <- na_summary %>%
#     mutate(year = lubridate::year(annualreportdate)) %>%
#     group_by(companyid, year) %>%
#     mutate(n = n_distinct(annualreportdate)) %>%
#     filter(n == 1) %>%
#     ungroup() %>%
#     select(-year, - n)

# n_distinct(select(na_summary, isin, annualreportdate))

That would remove 1,270 observations.

### Duplicate observations of directors

In [16]:
na_summary %>%
    group_by(directorid, boardid, annualreportdate) %>%
    summarize(n = n()) %>%
    {table(.$n)}

`summarise()` regrouping output by 'directorid', 'boardid' (override with `.groups` argument)




     1      2      3      4      5      6      8     10 
678428  49254   6309   1666    602    362    141      1 

For some reasons, there are a lot of instances of duplicate observations for directors.

In [17]:
na_summary_original %>%
    group_by(directorid, boardid, annualreportdate) %>%
    mutate(n = n()) %>%
    arrange(desc(n), annualreportdate) %>%
    select(directorid, annualreportdate, everything()) %>%
    head()

directorid,annualreportdate,rowtype,boardname,ned,directorname,rolename,rolestatus,gender,nationality,⋯,numberdirectors,stdevtimebrd,stdevtimeinco,stdevtotnolstdbrd,stdevtotcurrnolstdbrd,stdevnoquals,stdevage,networksize,companyid,n
<dbl>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
33628,2013-12-01,Board Member,LIBERTY MEDIA CORP,No,Greg Maffei,President/CEO,Greg Maffei joined this role on 14 Jan 2013,M,American,⋯,9,0,0,7.9,2.3,1.3,11.7,4646,1924862,16
33628,2013-12-01,Board Member,LIBERTY MEDIA CORP,No,Greg Maffei,President/CEO,Greg Maffei joined this role on 14 Jan 2013,M,American,⋯,9,0,0,7.9,2.3,1.3,11.7,4646,1924862,16
33628,2013-12-01,Board Member,LIBERTY MEDIA CORP,No,Greg Maffei,President/CEO,Greg Maffei joined this role on 14 Jan 2013,M,American,⋯,9,0,0,7.9,2.3,1.3,11.7,4646,1924862,16
33628,2013-12-01,Disclosed Earner,LIBERTY MEDIA CORP,No,Greg Maffei,President/CEO,Greg Maffei joined this role on 14 Jan 2013,M,American,⋯,9,0,0,7.9,2.3,1.3,11.7,4646,1924862,16
33628,2013-12-01,Board Member,LIBERTY MEDIA CORP,No,Greg Maffei,President/CEO,Greg Maffei joined this role on 14 Jan 2013,M,American,⋯,9,0,0,7.9,2.3,1.3,11.7,4646,1924862,16
33628,2013-12-01,Disclosed Earner,LIBERTY MEDIA CORP,No,Greg Maffei,President/CEO,Greg Maffei joined this role on 14 Jan 2013,M,American,⋯,9,0,0,7.9,2.3,1.3,11.7,4646,1924862,16


At least in this case, it seems that the entries are just duplicates. Some of the duplicates seem to be the same person taking different roles.

### Validate other columns

#### Gender variable

In [18]:
table(na_summary$gender) 


     F      M 
 89777 719070 

The data does not contain any values other than F and M.

In [19]:
na_summary %>%
    group_by(directorid) %>%
    summarize(n = n_distinct(gender)) %>%
    arrange(desc(n)) %>%
    head(3)

`summarise()` ungrouping output (override with `.groups` argument)



directorid,n
<dbl>,<int>
1,1
16,1
27,1


All directors have the same gender across all reports, that's also good to know.

#### boardid vs companyid

In [20]:
table(na_summary_original$boardid == na_summary_original$companyid)


   TRUE 
1322206 

Interesting

#### Validate cusip and ISIN

Length for ISIN should always be 12: country code + cusip + check digit.

In [21]:
table(nchar(na_summary$isin))


    12 
808847 

That's good.

Make sure that compa always provides 9 char cusip.

In [22]:
compa_cusip <- read_feather("data/compa.feather")$cusip
invisible(gc())

In [23]:
table(nchar(compa_cusip))


     9 
803402 

See how great the overlap is between the two.

In [24]:
table(stringr::str_sub(na_summary$isin, 3, 11) %in% compa_cusip)


 FALSE   TRUE 
122666 686181 

That's a pretty good coverage.

## Board characteristics dataset

In [25]:
board <- read_feather("data/na_boards.feather")
head(board)

boardname,annualreportdate,committeename,directorname,committeerolename,functionalexperience,boardrole,brdposition,dirrolestatus,boardid,clientcompanyid,ned,directorid,clientdirectorid
<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<???>,<chr>,<dbl>,<???>
1 800 CONTACTS INC (De-listed 09/2007),2006-12-01,Compensation,Steve Key,Member,NA,Independent Director,Yes,Steve Key joined this role in 28 Jul 2005,3,NA,Yes,59796,NA
1 800 CONTACTS INC (De-listed 09/2007),2006-12-01,Audit,Steve Key,Chairman / Financial Expert,Executive VP/CFO(2),Independent Director,Yes,Steve Key joined this role in 02 Aug 2005,3,NA,Yes,59796,NA
1 800 CONTACTS INC (De-listed 09/2007),2006-12-01,Audit,Steve Key,Chairman / Financial Expert,Vice Chairman/CFO(1),Independent Director,Yes,Steve Key joined this role in 02 Aug 2005,3,NA,Yes,59796,NA
1 800 CONTACTS INC (De-listed 09/2007),2005-12-01,Compensation,Steve Key,Member,NA,Independent Director,Yes,Steve Key joined this role in 28 Jul 2005,3,NA,Yes,59796,NA
1 800 CONTACTS INC (De-listed 09/2007),2005-12-01,Audit,Steve Key,Chairman / Financial Expert,Executive VP/CFO(2),Independent Director,Yes,Steve Key joined this role in 02 Aug 2005,3,NA,Yes,59796,NA
1 800 CONTACTS INC (De-listed 09/2007),2005-12-01,Audit,Steve Key,Chairman / Financial Expert,Vice Chairman/CFO(1),Independent Director,Yes,Steve Key joined this role in 02 Aug 2005,3,NA,Yes,59796,NA


In [26]:
nrow(board)

nrow(unique(board))

nrow(unique(select(board, -c(functionalexperience, ned, boardrole, brdposition))))

board %>%
    select(boardid, directorid, annualreportdate, committeename) %>%
    unique() %>%
    {nrow(.)}

[1] 1821999

[1] 1821992

[1] 1524147

[1] 1524147

There are some duplicates with diverging data, but only in variables that are not relevant.

## Compa dataset

In [27]:
compa <- read_feather("data/compa_preprocessed.feather")
head(compa)

gvkey,fyear,cusip,apdedate,at,emp,dltt,ceq,act,lct,bkvlps,csho
<chr>,<dbl>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
001000,1961,000032102,NA,NA,NA,0.100,NA,NA,NA,2.4342,0.152
001000,1962,000032102,NA,NA,NA,0.000,0.552,NA,NA,3.0497,0.181
001000,1963,000032102,NA,NA,NA,0.015,0.553,0.408,0.322,2.9731,0.186
001000,1964,000032102,NA,1.416,NA,0.522,0.607,0.718,0.267,3.0969,0.196
001000,1965,000032102,NA,2.310,NA,1.154,0.491,0.725,0.623,2.3835,0.206
001000,1966,000032102,NA,2.430,NA,1.109,0.834,1.015,0.446,3.8082,0.219


In [28]:
compa %>%
    group_by(cusip, fyear) %>%
    mutate(n = n()) %>%
    filter(n > 1) %>%
    head(10)

gvkey,fyear,cusip,apdedate,at,emp,dltt,ceq,act,lct,bkvlps,csho,n
<chr>,<dbl>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
001004,2006,000361105,2007-05-31,1067.633,3.90,253.611,494.243,645.721,256.506,13.0998,37.729,2
001004,2006,000361105,NA,NA,NA,NA,NA,NA,NA,NA,NA,2
001004,2010,000361105,2011-05-31,1703.727,6.10,329.802,835.845,913.985,416.010,21.0112,39.781,2
001004,2010,000361105,NA,NA,NA,NA,NA,NA,NA,NA,NA,2
001004,2011,000361105,2012-05-31,2195.653,6.70,669.489,864.649,1063.272,473.226,21.4697,40.273,2
001004,2011,000361105,NA,NA,NA,NA,NA,NA,NA,NA,NA,2
001004,2013,000361105,2014-05-31,2199.500,5.80,564.300,999.500,1116.900,402.100,25.2654,39.560,2
001004,2013,000361105,NA,2159.800,NA,NA,NA,NA,NA,NA,NA,2
001004,2014,000361105,2015-05-31,1515.000,4.85,85.000,845.100,954.100,412.000,23.8574,35.423,2


There is a weird thing going on here, where there are many almost empty observations that we can drop.

In [29]:
compa_cleaned <- read_feather("data/compa_cleaned.feather")
compa_cleaned %>%
    group_by(gvkey, fyear) %>%
    mutate(n = n()) %>%
    filter(n > 1) %>%
    arrange(desc(gvkey, fyear))

gvkey,fyear,cusip,apdedate,at,emp,dltt,ceq,act,lct,bkvlps,csho,n
<chr>,<dbl>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>


That method allowed us to clean up compa.